### ⛔ Causal (Masked) Attention

When building **contextualized embeddings**, standard self-attention allows *every token* to attend to *every other token* — including **future tokens**.

But wait... 👀  
Doesn't that create a **bias**?  
Yes! Because when predicting the next word, the model **should not peek** at words that come *after* it.

---

#### 💭 Example

Consider the sentence:

> "Hey how are you?"

If we allow full attention, then:

- Embedding of **"Hey"** = depends on *"Hey", "how", "are", "you"*  
- Embedding of **"How"** = depends on *"Hey", "how", "are", "you"*  
- Embedding of **"are"** = depends on *"Hey", "how", "are", "you"*  
- Embedding of **"you"** = depends on *"Hey", "how", "are", "you"*

That means even the **first word** already “knows” all the **future words**, which is *unrealistic* for language generation.

---

#### 🧠 Correct Behavior (Causal Masking)

To fix this, we use **causal attention** (also known as **masked self-attention**).  
It ensures that each token can only attend to **previous** and **current** tokens — not future ones.

$$
\text{Allowed attention: only to past and current positions.}
$$

So now:

| Token | Can Attend To | Masked Tokens |
|:------|:---------------|:--------------|
| **"Hey"** | "Hey" | "how", "are", "you" |
| **"How"** | "Hey", "How" | "are", "you" |
| **"are"** | "Hey", "How", "are" | "you" |
| **"you"** | "Hey", "How", "are", "you" | — |

---

#### ⚙️ How It Works

Before applying **softmax** on the attention scores,  
we apply a **mask** on all *future positions* (upper triangular part of the matrix):

$$
\text{mask}[i, j] =
\begin{cases}
0 & \text{if } j \leq i \\
-\infty & \text{if } j > i
\end{cases}
$$

Then:
$$
\text{Attention} = \text{softmax}\left( \frac{QK^T}{\sqrt{d_k}} + \text{mask} \right) V
$$

This forces all future tokens (where \( j > i \)) to have **zero attention probability** after softmax.

---

#### 🚀 In Summary

- **Problem:** Standard attention lets words see the future (data leakage).  
- **Solution:** Causal masking blocks future tokens.  
- **Effect:** The model becomes **autoregressive** — it learns to predict the *next token* using only *past* context.  
- **Used in:** GPT, GPT-2, GPT-3, GPT-4, and all autoregressive LLMs.


In [19]:
# lets take a input sentence
import torch
context_len = 4
embed_dim = 5

inputs = torch.rand(4, 5)
# 4 words each with embedding dim of 5 
inputs

tensor([[0.3714, 0.4833, 0.1388, 0.0561, 0.0588],
        [0.2616, 0.0533, 0.2163, 0.4509, 0.6847],
        [0.2808, 0.2020, 0.4746, 0.1538, 0.7563],
        [0.5564, 0.0253, 0.7019, 0.7511, 0.6994]])

In [20]:
# will define query, key, value weight matrices
Wq = torch.nn.Parameter(torch.rand(embed_dim, embed_dim))
Wk = torch.nn.Parameter(torch.rand(embed_dim, embed_dim))
Wv = torch.nn.Parameter(torch.rand(embed_dim, embed_dim))

In [21]:
# query, key and value embeddings
queries = inputs @ Wq 
keys = inputs @ Wk 
values = inputs @ Wv 

print(f"Shape of queries : {queries.shape}")

Shape of queries : torch.Size([4, 5])


In [22]:
# now we'll find attention scores 
attn_scores = queries @ keys.T # should be a 4x4 matrix for our use case
attn_scores

tensor([[1.8263, 2.2394, 2.7503, 3.8324],
        [2.2149, 2.8085, 3.4401, 4.7019],
        [2.8050, 3.5435, 4.3073, 5.9037],
        [3.9791, 5.0077, 6.0843, 8.3803]], grad_fn=<MmBackward0>)

In [23]:
# now we want will make a mask that when applied to attn_scores then upper half become -infinity, so that when we take softmax those becomes 0

mask = torch.triu(torch.ones(context_len, context_len), diagonal=1)
mask

tensor([[0., 1., 1., 1.],
        [0., 0., 1., 1.],
        [0., 0., 0., 1.],
        [0., 0., 0., 0.]])

In [24]:
# we'll multiply it by -infinity 
attn_scores_masked = attn_scores.masked_fill(mask.bool(), -torch.inf) / torch.sqrt(torch.tensor(embed_dim, dtype=torch.float32))
attn_scores_masked

tensor([[0.8167,   -inf,   -inf,   -inf],
        [0.9906, 1.2560,   -inf,   -inf],
        [1.2544, 1.5847, 1.9263,   -inf],
        [1.7795, 2.2395, 2.7210, 3.7478]], grad_fn=<DivBackward0>)

In [25]:
# apply softmax 
attn_scores_scaled = torch.softmax(attn_scores_masked, dim=-1)
attn_scores_scaled

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.4340, 0.5660, 0.0000, 0.0000],
        [0.2299, 0.3199, 0.4502, 0.0000],
        [0.0813, 0.1287, 0.2083, 0.5817]], grad_fn=<SoftmaxBackward0>)

In [26]:
# use these attention weight to calculate contextualized words
contextualixed_inputs = attn_scores_scaled @ values
contextualixed_inputs

tensor([[1.0175, 0.4273, 0.4822, 0.4923, 0.4856],
        [1.2470, 0.8506, 0.5102, 0.7445, 0.8242],
        [1.4388, 1.0627, 0.5218, 0.8687, 0.9431],
        [1.9589, 1.5466, 0.8183, 1.3098, 1.4762]], grad_fn=<MmBackward0>)

#### Causal Attention Class

In [68]:
import torch.nn as nn 

class CausalAttention(nn.Module):
    def __init__(self, dim, context_len, dropout=0.1):
        super().__init__()
        self.Wq = torch.nn.Parameter(torch.rand(dim, dim))
        self.Wk = torch.nn.Parameter(torch.rand(dim, dim))
        self.Wv = torch.nn.Parameter(torch.rand(dim, dim))
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer( 'mask',torch.triu(torch.ones(context_len, context_len), diagonal=1))

    def forward(self, x):
        # x is input senetence (batch_size, context_len, embedding_dim)
        # calculate key, query and value
        batch_size, context_len, emb_dim = x.shape
        keys = x @ self.Wk
        queries = x @ self.Wq 
        values = x @ self.Wv 

        # attention scores 
        attn_scores = queries @ keys.transpose(1, 2)
        # scaling attention scores 
        # mask = torch.triu(torch.ones(x.shape[0], x.shape[0]), diagonal=1)
        attn_scores = attn_scores.masked_fill_(self.mask.bool()[:context_len, :context_len], -torch.inf)

        attn_scores = attn_scores / torch.sqrt(torch.tensor(x.shape[1], dtype=torch.float32))

        scaled_attn_weights = torch.softmax(attn_scores, dim=-1)
        scaled_attn_weights = self.dropout(scaled_attn_weights)
        contextualized_inputs = scaled_attn_weights @ values 
        return contextualized_inputs

In [69]:
causal_attention = CausalAttention(4, 6)
inputs2 = torch.rand(6, 4)
inputs2

tensor([[0.4855, 0.6808, 0.0109, 0.0573],
        [0.9897, 0.2111, 0.7847, 0.7631],
        [0.0547, 0.4976, 0.6631, 0.6122],
        [0.9288, 0.4945, 0.2420, 0.9873],
        [0.1955, 0.6456, 0.5109, 0.1696],
        [0.7426, 0.5118, 0.6644, 0.7572]])

In [70]:
## Now we'll process the bacth of inputs (2 sentences)
batch = torch.stack((inputs2, inputs2), dim=0)
batch.shape

torch.Size([2, 6, 4])

In [71]:
# after applying causal attention the shape should be same 
batch_output = causal_attention(batch)
batch_output.shape

torch.Size([2, 6, 4])